In [2]:
import torch
import os
import sys
base_path = os.path.dirname(__file__) if '__file__' in globals() else os.getcwd()
sys.path.append(os.path.abspath(os.path.join(base_path, 'OpenVoice')))
print(sys.executable)
from OpenVoice.openvoice import se_extractor
from OpenVoice.openvoice.api import BaseSpeakerTTS, ToneColorConverter
import pygame as pg
import io

c:\Users\minhq\voice-assistance\xtts_env\Scripts\python.exe


c:\Users\minhq\voice-assistance\xtts_env\lib\site-packages\jieba\_compat.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
c:\Users\minhq\voice-assistance\xtts_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.

pygame 2.6.1 (SDL 2.28.4, Python 3.10.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [7]:
ckpt_base='OpenVoice/checkpoints/base_speakers/EN'
ckpt_converter = 'OpenVoice/checkpoints/converter'
output_dir = 'OpenVoice/outputs'
reference_speaker = 'OpenVoice/resources/demo_speaker2.mp3'
text = """Energy Secretary Chris Wright, who made millions in the fracking industry, commissioned the report. In a preface, he did not deny that climate change exists.
    “Climate change is real, and it deserves attention,” he wrote. “But it is not the greatest threat facing humanity. That distinction belongs to global energy poverty.”
    In other words, Wright sees more damage to humans from cutting back on carbon emissions.
    That is a minority view in the scientific community, which has a much, much larger body of peer reviewed studies that raise the alarm about climate change. Most notably, the Intergovernmental Panel on Climate Change issues peer-reviewed reports with hundreds of authors from around world. The Trump administration has barred US government scientists from taking part in the next installment, due out in 2029."""
device = "cuda" if torch.cuda.is_available else "cpu"


In [10]:
base_speaker_tts = BaseSpeakerTTS(f'{ckpt_base}/config.json', device=device)
base_speaker_tts.load_ckpt(f'{ckpt_base}/checkpoint.pth')
tone_color_converter = ToneColorConverter(f'{ckpt_converter}/config.json', device=device)
tone_color_converter.load_ckpt(f'{ckpt_converter}/checkpoint.pth')
source_se = torch.load(f'{ckpt_base}/en_default_se.pth').to(device)

c:\Users\minhq\voice-assistance\xtts_env\lib\site-packages\torch\nn\utils\weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Loaded checkpoint 'OpenVoice/checkpoints/base_speakers/EN/checkpoint.pth'
missing/unexpected keys: [] []
Loaded checkpoint 'OpenVoice/checkpoints/converter/checkpoint.pth'
missing/unexpected keys: [] []


In [13]:
target_se, audio_name = se_extractor.get_se(reference_speaker,
                                            tone_color_converter,
                                            target_dir='processed', 
                                            vad=True)

OpenVoice version: v1
[(0.0, 8.21), (9.39, 12.946), (13.262, 29.49225)]
after vad: dur = 27.995986394557825


c:\Users\minhq\voice-assistance\xtts_env\lib\site-packages\torch\functional.py:730: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\SpectralOps.cpp:880.)
  return _VF.stft(  # type: ignore[attr-defined]


In [14]:
save_path = f'{output_dir}/output_en_default.wav'
src_path = f'{output_dir}/tmp.wav'
buffer_ouput = base_speaker_tts.tts(text, src_path, speaker="excited", language='English', speed=0.9)
encode_message = "@Myshell"
tone_color_converter.convert(
    audio_src_path=src_path, 
    src_se=source_se, 
    tgt_se=target_se, 
    output_path=save_path,
    message=encode_message)

 > Text splitted to sentences.
Energy Secretary Chris Wright, who made millions in the fracking industry,
commissioned the report. In a preface, he did not deny that climate change exists.
Climate change is real, and it deserves attention, he wrote. But it is not the greatest threat facing humanity.
That distinction belongs to global energy poverty. In other words, Wright sees more damage to humans from cutting back on carbon emissions.
That is a minority view in the scientific community, which has a much,
much larger body of peer reviewed studies that raise the alarm about climate change.
Most notably, the Intergovernmental Panel on Climate Change issues peer-reviewed reports with hundreds of authors from around world.
The Trump administration has barred US government scientists from taking part in the next installment,
due out in 2029.
 > ===========================
ˈɛnəɹdʒi ˈsɛkɹəˌtɛɹi kɹɪs ɹaɪt, hu meɪd ˈmɪljənz ɪn ðə fɹacking* ˈɪndəstɹi,
 length:75
 length:74
Voice that we can use

In [ ]:
def play_audio(src_path):
    pg.init()
    pg.mixer.init()
    pg.mixer.music.load(src_path, "wav")
    pg.mixer.music.play()
    while pg.mixer.music.get_busy():  # Wait for playback to finish
        pass
play_audio(buffer_ouput)

***TTS with xTTS***

In [3]:
import TTS, torch, torchaudio
print(TTS.__version__, torch.__version__, torchaudio.__version__)
import huggingface_hub
print(huggingface_hub.__version__)
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts

0.21.1 2.8.0+cu126 2.8.0+cu126
0.25.2


In [ ]:
xtts_config = XttsConfig()
xtts_config.load_json("xTTS/config.json")

#Initialize XTTS model
xtts_model = Xtts.init_from_config(xtts_config)
# xtts_model.load_checkpoint(xtts_config, checkpoint_path="xTTS/model.pth",vocab_path="xTTS/vocab.json", eval=True)
with torch.serialization.safe_globals([XttsConfig]):
    checkpoint = torch.load("xTTS/model.pth", map_location="cpu", weights_only=False)
xtts_model.load_state_dict(checkpoint['model'])

from tokenizers import Tokenizer
import json

vocab_path = 'xTTS/vocab.json'
with open(vocab_path, "r", encoding="utf-8") as f:
    vocab_data = json.load(f)

# Create a tokenizer from vocab.json
# This assumes vocab.json follows HuggingFace's tokenizer format
tokenizer = Tokenizer.from_file(vocab_path)
xtts_model.tokenizer = tokenizer
xtts_model.cuda()

Xtts(
  (gpt): GPT(
    (conditioning_encoder): ConditioningEncoder(
      (init): Conv1d(80, 1024, kernel_size=(1,), stride=(1,))
      (attn): Sequential(
        (0): AttentionBlock(
          (norm): GroupNorm32(32, 1024, eps=1e-05, affine=True)
          (qkv): Conv1d(1024, 3072, kernel_size=(1,), stride=(1,))
          (attention): QKVAttention()
          (x_proj): Identity()
          (proj_out): Conv1d(1024, 1024, kernel_size=(1,), stride=(1,))
        )
        (1): AttentionBlock(
          (norm): GroupNorm32(32, 1024, eps=1e-05, affine=True)
          (qkv): Conv1d(1024, 3072, kernel_size=(1,), stride=(1,))
          (attention): QKVAttention()
          (x_proj): Identity()
          (proj_out): Conv1d(1024, 1024, kernel_size=(1,), stride=(1,))
        )
        (2): AttentionBlock(
          (norm): GroupNorm32(32, 1024, eps=1e-05, affine=True)
          (qkv): Conv1d(1024, 3072, kernel_size=(1,), stride=(1,))
          (attention): QKVAttention()
          (x_proj): Ide

In [43]:
import soundfile as sf
sample_audio = "OpenVoice/resources/example_reference.mp3"
print(os.path.exists(sample_audio))
print(os.path.abspath(sample_audio))
print(xtts_model)
import traceback
def process_and_play(xtts_model, prompt, audio_file_pth): 
    try:
        # Use XTTS to synthesize speech
        outputs = xtts_model.synthesize(
            prompt,
            xtts_config,
            speaker_wav=audio_file_pth, # Pass the file path directly
            gpt_cond_len=24,
            temperature=0.6,
            language='en',
        )
        synthesized_audio = outputs['wav']
        print(synthesized_audio)
# Save the synthesized audio to the output path 
        src_path = f'{output_dir}/output.wav' 
        sample_rate = xtts_config.audio.sample_rate 
        sf.write(src_path, synthesized_audio, sample_rate)
        print("Audio generated successfully.")
        play_audio(src_path)
    except Exception as e:
        print(traceback.format_exc())
        print (f"Error during audio generation: {e}")

True
c:\Users\minhq\voice-assistance\OpenVoice\resources\example_reference.mp3
Xtts(
  (gpt): GPT(
    (conditioning_encoder): ConditioningEncoder(
      (init): Conv1d(80, 1024, kernel_size=(1,), stride=(1,))
      (attn): Sequential(
        (0): AttentionBlock(
          (norm): GroupNorm32(32, 1024, eps=1e-05, affine=True)
          (qkv): Conv1d(1024, 3072, kernel_size=(1,), stride=(1,))
          (attention): QKVAttention()
          (x_proj): Identity()
          (proj_out): Conv1d(1024, 1024, kernel_size=(1,), stride=(1,))
        )
        (1): AttentionBlock(
          (norm): GroupNorm32(32, 1024, eps=1e-05, affine=True)
          (qkv): Conv1d(1024, 3072, kernel_size=(1,), stride=(1,))
          (attention): QKVAttention()
          (x_proj): Identity()
          (proj_out): Conv1d(1024, 1024, kernel_size=(1,), stride=(1,))
        )
        (2): AttentionBlock(
          (norm): GroupNorm32(32, 1024, eps=1e-05, affine=True)
          (qkv): Conv1d(1024, 3072, kernel_size=(

In [44]:
text2 = "Hello, I am a ML developer"
# Convert to WAV if needed
if sample_audio.endswith(".mp3"):
    wav_path = sample_audio.replace(".mp3", ".wav")
    data, sr = sf.read(sample_audio)
    sf.write(wav_path, data, sr)
    speaker_path = wav_path
process_and_play(xtts_model=xtts_model, prompt=text2, audio_file_pth=sample_audio)

Traceback (most recent call last):
  File "C:\Users\minhq\AppData\Local\Temp\ipykernel_20208\1801340675.py", line 10, in process_and_play
    outputs = xtts_model.synthesize(
  File "c:\Users\minhq\voice-assistance\xtts_env\lib\site-packages\TTS\tts\models\xtts.py", line 392, in synthesize
    return self.inference_with_config(text, config, ref_audio_path=speaker_wav, language=language, **kwargs)
  File "c:\Users\minhq\voice-assistance\xtts_env\lib\site-packages\TTS\tts\models\xtts.py", line 414, in inference_with_config
    return self.full_inference(text, ref_audio_path, language, **settings)
  File "c:\Users\minhq\voice-assistance\xtts_env\lib\site-packages\torch\utils\_contextlib.py", line 120, in decorate_context
    return func(*args, **kwargs)
  File "c:\Users\minhq\voice-assistance\xtts_env\lib\site-packages\TTS\tts\models\xtts.py", line 483, in full_inference
    return self.inference(
  File "c:\Users\minhq\voice-assistance\xtts_env\lib\site-packages\torch\utils\_contextlib.p

***Faster whisper***

In [ ]:
from faster_whisper import WhisperModel
model_size = "small.en"
local_dir = "STT-model"
import os
if not os.path.exists(local_dir):
    os.mkdir(local_dir)
    whisper_model = WhisperModel(model_size, device="cpu", compute_type="int8", download_root=local_dir)
else:
    whisper_model = WhisperModel('STT-model\models--guillaumekln--faster-whisper-small.en\snapshots\model', device="cpu", compute_type="int8", local_files_only=True)
import time

In [ ]:
def transcribe_audio(audio_path):
    start_time = time.time()
    segments, info = whisper_model.transcribe(audio_path, beam_size=5)
    transcription = ""
    for segment in segments:
        transcription += segment.text + " "
    end_time = time.time()
    inference_time = (end_time-start_time)
    print(f"Inference time: {inference_time}")
    return transcription.strip()

path = "OpenVoice/resources/example_reference.wav"
output = transcribe_audio(path)
print(output)

: 

***So the TTS model running oke, now we test the LLM model***

In [1]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as f

c:\Users\minhq\voice-assistance\xtts_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


***Testing with model MiniLM-L6 by Sentence-tranformers, and it's used just for sentence encoding only***

In [ ]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

model_name = 'sentence-transformers/all-MiniLM-L6-v2'
cache_dir = 'MiniLM-L6'
# Sentences we want sentence embeddings for
sentences = ['Each sentence is converted']

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=cache_dir)
model = AutoModel.from_pretrained(model_name, cache_dir=cache_dir)

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = f.normalize(sentence_embeddings, p=2, dim=1)

# the sentence embeddings return a 2D dimentional array
print("Sentence embeddings:")
print(sentence_embeddings)

# decoded_text  = tokenizer.decode(encoded_input['input_ids'][0], skip_special_tokens=True)
# print(f"Original sentence: {decoded_text}")

c:\Users\minhq\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\modeling_utils.py:415: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(

Sentence embeddings:
tensor([[ 8.6439e-02,  1.0276e-01,  5.3946e-03,  2.0444e-03, -9.9633e-03,
          2.5386e-02,  4.9288e-02, -3.0627e-02,  6.8725e-02,  1.0137e-02,
          7.7540e-02, -9.0081e-02,  6.1062e-03, -5.6990e-02,  1.4171e-02,
          2.8049e-02, -8.6846e-02,  7.6440e-02, -1.0349e-01, -6.7744e-02,
          6.9995e-02,  8.4425e-02, -7.2491e-03,  1.0477e-02,  1.3402e-02,
          6.7758e-02, -9.4209e-02, -3.7169e-02,  5.2262e-02, -3.1085e-02,
         -9.6341e-02,  1.5772e-02,  2.5787e-02,  7.8525e-02,  7.8995e-02,
          1.9152e-02,  1.6436e-02,  3.1009e-03,  3.8131e-02,  2.3709e-02,
          1.0539e-02, -4.4064e-02,  4.4174e-02, -2.5873e-02,  6.1538e-02,
         -4.0543e-02, -8.6414e-02,  3.1972e-02, -8.9067e-04, -2.4444e-02,
         -9.1972e-02,  2.3394e-02, -8.3029e-02,  4.4151e-02, -2.4969e-02,
          6.2302e-02, -1.3036e-03,  7.5140e-02,  2.4638e-02, -6.4724e-02,
         -1.1773e-01,  3.8339e-02, -9.1177e-02,  6.3545e-02,  7.6274e-02,
         -8.8024e

***Now we test the LLM model***. We can run different models with different sizes, for example 6B, or larger liek 20B, or 100B parameters. But everything must start with "checking device" first, choosing what models to run depends on the hardware configuration. Some PC only has CPU supported while others have strong GPU to run heavy model.

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from accelerate import init_empty_weights, infer_auto_device_map
import warnings

warnings.filterwarnings("ignore", message=".*copying from a non-meta parameter.*")
model_name = "facebook/opt-2.7B"
cache_dir = "./optimized-opt-2.7B-8bit"
local_dir = "./optimized-opt-2.7B-8bit/models--facebook--opt-2.7B/snapshots/905a4b602cda5c501f1b3a2650a4152680238254"

tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=cache_dir)

# Initialize model with empty weights to manage device map efficiently
with init_empty_weights():
    model = AutoModelForCausalLM.from_pretrained(model_name, cache_dir=cache_dir)
    # tie the model weights to prevent memory overhead
    model.tie_weights()

# Infer device map to manage model layers between GPU and CPU
device_map = infer_auto_device_map(model, max_memory={0: "4GiB", "cpu": "6GiB"})

# configure quantization using BitsAndBytesConfig
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_enable_fp32_cpu_offload=True
)

# Load the model with 8-bit quantization and mixed precision
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    cache_dir=cache_dir,
    device_map="auto",
    quantization_config=quantization_config,
    offload_folder="./offload",
    torch_dtype=torch.float16,
    offload_state_dict=True
)

# Enable gradient checkpointing
model.gradient_checkpointing_enable()

def generate_text(prompt, max_length=500):
    # Move inputs to the appropriate device (CPU or GPU)
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")

    # Generate text using the optimized model
    with torch.no_grad():
        outputs = model.generate(
            inputs.input_ids,
            max_length=max_length,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=0.7
        )

    # Decode the outputs and return the generated text
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

prompt = "Once upon a time"
generated_text = generate_text(prompt)
print(generated_text)

c:\Users\minhq\voice-assistance\xtts_env\lib\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\minhq\voice-assistance\xtts_env\lib\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


ImportError: Using `load_in_8bit=True` requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes `pip install -i https://test.pypi.org/simple/ bitsandbytes` or pip install bitsandbytes` 